생각보다 오버피팅이 잘 일어나는 듯 조심해야함!

# Data Set

In [1]:
import warnings
warnings.filterwarnings('ignore')

import koreanize_matplotlib

In [2]:
import pandas as pd

train = pd.read_csv('Data/train_0820.csv')
test = pd.read_csv('Data/test_0820.csv')

In [52]:
train['건물번호'] = train['건물번호'].astype('category')
test['건물번호'] = test['건물번호'].astype('category')

train['weekday'] = train['weekday'].astype('category')
test['weekday'] = test['weekday'].astype('category')

# btype_ 으로 시작하는 모든 컬럼 제거
train = train.drop(columns=[c for c in train.columns if c.startswith("btype_")], errors="ignore")
test  = test.drop(columns=[c for c in test.columns if c.startswith("btype_")], errors="ignore")

test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16800 entries, 0 to 16799
Data columns (total 40 columns):
 #   Column              Non-Null Count  Dtype   
---  ------              --------------  -----   
 0   num_date_time       16800 non-null  object  
 1   건물번호                16800 non-null  category
 2   일시                  16800 non-null  object  
 3   기온(°C)              16800 non-null  float64 
 4   강수량(mm)             16800 non-null  float64 
 5   풍속(m/s)             16800 non-null  float64 
 6   습도(%)               16800 non-null  float64 
 7   건물유형                16800 non-null  object  
 8   연면적(m2)             16800 non-null  float64 
 9   냉방면적(m2)            16800 non-null  float64 
 10  태양광용량(kW)           16800 non-null  float64 
 11  ESS저장용량(kWh)        16800 non-null  float64 
 12  PCS용량(kW)           16800 non-null  float64 
 13  month               16800 non-null  int64   
 14  day                 16800 non-null  int64   
 15  hour                16800 non-null  

In [5]:
import random
import numpy as np
import os

In [53]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Seed 고정

In [54]:
train['건물유형'] = train['건물유형'].astype('category')
train_x = train.drop(['num_date_time', '일시', '일사(MJ/m2)', '일조(hr)', '전력소비량(kWh)'],axis = 1).iloc[:,:-10]
train_y = train['전력소비량(kWh)']

In [55]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(train_x, train_y, test_size=0.2, random_state=42)

In [56]:
X_train.shape, X_val.shape, y_train.shape, y_val.shape

((163200, 28), (40800, 28), (163200,), (40800,))

In [57]:
import numpy as np

def smape(y_true, y_pred):
    """
    SMAPE (Symmetric Mean Absolute Percentage Error)
    y_true: 실제값 (array-like)
    y_pred: 예측값 (array-like)
    반환값: SMAPE (단위: %)
    """
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    denominator = (np.abs(y_true) + np.abs(y_pred))
    diff = np.abs(y_pred - y_true)
    # 0으로 나누는 경우는 diff=0 이므로 0으로 처리됨
    smape_val = 100 * np.mean(2 * diff / np.where(denominator == 0, 1, denominator))
    return smape_val


## CatBoost 적용

In [58]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor, Pool

# --- 데이터 준비 (이미 train, test, sample_submission이 있다면 생략) ---
# train = pd.read_csv("/mnt/data/train_0820.csv")
# test  = pd.read_csv("/mnt/data/test_0820.csv")
# sub   = pd.read_csv("/mnt/data/sample_submission.csv")

TARGET = "전력소비량(kWh)"

def smape(y_true, y_pred):
    denom = (np.abs(y_true) + np.abs(y_pred))
    denom = np.where(denom == 0, 1, denom)
    return np.mean(2.0 * np.abs(y_pred - y_true) / denom) * 100

building_types = train['건물유형'].unique()
val_scores = {}
models = {}
preds_list = []


In [59]:
# train/test에서 동일하게 제거할 컬럼(문자열 시간, 타깃, 학습에 불필요한 것)
BASE_DROP = ['num_date_time', '일시', '건물유형', '일사(MJ/m2)', '일조(hr)']

for btype in building_types:
    # 1) 해당 건물유형 subset
    train_sub = train[train['건물유형'] == btype].copy()
    test_sub  = test[test['건물유형'] == btype].copy()

    # 2) feature/target 분리
    drop_cols = BASE_DROP + [TARGET]
    X = train_sub.drop(columns=drop_cols, errors="ignore")
    y = train_sub[TARGET].astype(float)

    # 3) train/val split
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    # 4) 모델 정의 
    params = dict(
        loss_function="RMSE",   # ← 대문자 권장 ("MAE"도 OK)
        eval_metric="SMAPE",
        learning_rate=0.05,
        depth=8,
        l2_leaf_reg=3.0,
        random_seed=42,
        iterations=2500,
        od_type="Iter",
        od_wait=200,
        verbose=False,
        allow_writing_files=False
    )
    model = CatBoostRegressor(**params)
    
    # 5) 학습
    cat_cols = [c for c in X_train.columns if str(X_train[c].dtype) in ["object", "category"]]
    train_pool = Pool(X_train, y_train, cat_features=cat_cols)
    val_pool   = Pool(X_val,   y_val,   cat_features=cat_cols)
    
    model.fit(train_pool, eval_set=val_pool, use_best_model=True)
    
    # 6) Validation 성능
    y_val_pred = model.predict(val_pool)
    val_scores[btype] = smape(y_val.values, y_val_pred)
    
    # 8) test 예측
    X_test = test_sub.drop(columns=BASE_DROP, errors="ignore").reindex(columns=X.columns, fill_value=np.nan)
    test_pool = Pool(X_test, cat_features=cat_cols)
    test_sub = test_sub.copy()
    test_sub['pred'] = model.predict(test_pool)
    
    preds_list.append(test_sub[['num_date_time', '건물번호', 'pred']])


In [60]:
# 전체 예측 결과 합치기
final_preds = pd.concat(preds_list, axis=0).sort_values(['num_date_time', '건물번호'])

print("Validation SMAPE by type:", {k: round(v, 4) for k, v in val_scores.items()})
print("평균 SMAPE:", round(np.mean(list(val_scores.values())), 4))

Validation SMAPE by type: {'호텔': np.float64(6.1053), '상용': np.float64(2.4935), '병원': np.float64(3.0041), '학교': np.float64(3.0769), '건물기타': np.float64(5.5101), '아파트': np.float64(9.2572), '연구소': np.float64(4.9858), '백화점': np.float64(6.0462), 'IDC(전화국)': np.float64(1.1063), '공공': np.float64(5.3121)}
평균 SMAPE: 4.6897


In [25]:
# 저장
submit =  pd.read_csv('Data/sample_submission.csv')
submit_final = (
    submit
    .merge(final_preds, on=['num_date_time'], how='left')
    .drop(columns=['answer','건물번호'], axis = 1)                          # 기존 answer 버리고
    .rename(columns={'pred': 'answer'})                # pred → answer 로 변경
)
submit_final.to_csv(f'Submit/submit_0821catboost{np.mean(list(val_scores.values()))}.csv',index = False)
submit_final

,num_date_time,answer
0,1_20240825 00,4395.967644
1,1_20240825 01,4344.255931
2,1_20240825 02,3695.123593
3,1_20240825 03,3105.130003
4,1_20240825 04,2843.383108
...,...,...
16795,100_20240831 19,2774.644984
16796,100_20240831 20,2779.902769
16797,100_20240831 21,2722.759968
16798,100_20240831 22,3137.212195


### 현재 베스트 모델 기준으로 피쳐임포턴스 확인 - 제외하니 더 안좋아짐